In [ ]:
!pip install datasets==2.10.1
!pip install transformers==4.27.3
!pip install librosa
!pip install evaluate>=0.30
!pip install jiwer
!pip install gradio
!pip install audiomentations

In [ ]:
from datasets import load_dataset
from huggingface_hub import login
from datasets import interleave_datasets
import evaluate
import numpy as np
from datasets import Audio
from audiomentations import *

In [ ]:
login(token='')

In [ ]:
# часть датасетов была объединена, поэтому представление здесь может расходиться с представлением из раздела данных. исправлю.

dataset1 = load_dataset('aanosov/golos_train_part1', split='train', streaming=True, use_auth_token=True)
dataset2 = load_dataset('aanosov/golos_train_part2', split='train', streaming=True, use_auth_token=True)
dataset3 = load_dataset('aanosov/common_voice_ru_train', split='train', streaming=True, use_auth_token=True)
dataset4 = load_dataset('aanosov/sova_audiobooks_train', split='train', streaming=True, use_auth_token=True)


dataset5 = load_dataset('aanosov/youtube_train_part1', split='train', streaming=True, use_auth_token=True)

# print(list(dataset1.take(10)))

# как мы видим, сэмплы в датасете отсортированы по имени, а не по источнику
# имена - случайные последовательности так что нет никакой необходимости перемешивать датафрейм

# Combining datasets

In [5]:
# обхединенный датасет принимает веротяности, с которыми надо брать сэмплы из датасетов
# указываем их пропорционально длине датасетов

def get_probalities(array):
    return np.array(array)/np.sum(array)

probalities = get_probalities([412.8, 322.3, 206.7, 298, 150])

In [6]:
train = interleave_datasets([dataset1,
                             dataset2,
                             dataset3,
                             dataset4,
                             dataset5], 
                            probabilities=probalities, seed=123)

In [7]:
# for sample in list(train.take(10)):
#     print(sample)
#     print()

### Test dataset

In [ ]:
test = load_dataset('aanosov/youtube_control_001_aug', split='train', use_auth_token=True)

In [9]:
test = test.cast_column("audio", Audio(sampling_rate=16000))
augment_waveform = Clip(p=1)

In [10]:
def augment_dataset(batch):
    
    audio = batch["audio"]["array"]

    augmented_audio = augment_waveform(samples=audio, sample_rate=16000)

    batch["audio"]["array"] = augmented_audio

    return batch

In [ ]:
test = test.map(augment_dataset)

--

In [ ]:
from transformers import WhisperFeatureExtractor

feature_extractor = WhisperFeatureExtractor.from_pretrained("openai/whisper-small")

In [ ]:
from transformers import WhisperTokenizer

tokenizer = WhisperTokenizer.from_pretrained("openai/whisper-small", language="ru", task="transcribe")

In [14]:
from transformers import WhisperProcessor

processor = WhisperProcessor.from_pretrained("openai/whisper-small", language="ru", task="transcribe")

In [15]:
from datasets import Audio

train = train.cast_column("audio", Audio(sampling_rate=16000))
test = test.cast_column("audio", Audio(sampling_rate=16000))

In [16]:
def prepare_dataset(batch):
    audio = batch["audio"]
    
    batch["input_features"] = feature_extractor(audio["array"], sampling_rate=audio["sampling_rate"]).input_features[0]

    batch["labels"] = tokenizer(batch["answer"]).input_ids
    return batch

In [17]:
train = train.map(prepare_dataset)
test = test.map(prepare_dataset)

Map:   0%|          | 0/327 [00:00<?, ? examples/s]

In [18]:
augment_waveform = Compose([
    
    AddGaussianNoise(min_amplitude=0.005, max_amplitude=0.015, p=0.2),
    
    TimeStretch(min_rate=0.8, max_rate=1.25, p=0.2, leave_length_unchanged=False),
    
    PitchShift(min_semitones=-4, max_semitones=4, p=0.2),
    
    BandPassFilter(p=0.1),
    
    ClippingDistortion(p=0.1),
    
    AddBackgroundNoise(
    sounds_path="/kaggle/input/esc50-noises-train",
    min_snr_in_db=3.0,
    max_snr_in_db=30.0,
    noise_transform=PolarityInversion(),
    p=0.2),
    
    Gain(p=0.1),
    
    ApplyImpulseResponse(ir_path='/kaggle/input/impulseresponse-dataset', p=0.2), 
    
    Clip(p=1)
    ,])


def augment_dataset(batch):
    
    audio = batch["audio"]["array"]

    augmented_audio = augment_waveform(samples=audio, sample_rate=16000)

    batch["audio"]["array"] = augmented_audio

    return batch

In [19]:
train = train.map(augment_dataset)

In [20]:
import torch

from dataclasses import dataclass
from typing import Any, Dict, List, Union


class DataCollatorSpeechSeq2SeqWithPadding:
    
    def __init__(self, processor):
        self.processor = processor

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:

        input_features = [{"input_features": feature["input_features"]} for feature in features]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

        label_features = [{"input_ids": feature["labels"]} for feature in features]

        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        if (labels[:, 0] == self.processor.tokenizer.bos_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels

        return batch

In [21]:
data_collator = DataCollatorSpeechSeq2SeqWithPadding(processor=processor)

In [ ]:
metric_wer = evaluate.load("wer")
metric_cer = evaluate.load("cer")

In [23]:
# def normalize_string(x):
#     x = re.sub("[^0-9A-Za-zА-Яёа-яЁ ]", " ", x).lower()
#     return " ".join(x.split())

In [24]:
def compute_metrics(pred):
    pred_ids = pred.predictions
    label_ids = pred.label_ids

    # replace -100 with the pad_token_id
    label_ids[label_ids == -100] = tokenizer.pad_token_id

    # we do not want to group tokens when computing the metrics
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = tokenizer.batch_decode(label_ids, skip_special_tokens=True)

    wer = 100 * metric_wer.compute(predictions=pred_str, references=label_str)
    cer = 100 * metric_cer.compute(predictions=pred_str, references=label_str)

    return {"wer": wer, "cer": cer}

In [ ]:
from transformers import WhisperForConditionalGeneration

model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-small")

In [26]:
model.config.forced_decoder_ids = None
model.config.suppress_tokens = []

In [27]:
from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir="/kaggle/working/whisper-small",  # change to a repo name of your choice
    per_device_train_batch_size=2,
    gradient_accumulation_steps=8,  # increase by 2x for every 2x decrease in batch size
    learning_rate=5e-6,
    warmup_steps=1000,
    max_steps=10000,
    gradient_checkpointing=True,
    fp16=True,
    evaluation_strategy="steps",
    per_device_eval_batch_size=4,
    predict_with_generate=True,
    generation_max_length=400,
    save_steps=2000,
    eval_steps=500,
    report_to=["tensorboard"],
    load_best_model_at_end=True,
    metric_for_best_model="wer",
    greater_is_better=False,
    push_to_hub=False,
)

In [28]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=train,
    eval_dataset=test,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=processor.feature_extractor,
)

-----

In [ ]:
trainer.evaluate()

In [29]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
trainer.train()